In [ ]:
# Installing the dependencies
pip install --upgrade-strategy eager install optimum[onnxruntime]

In [ ]:
# Import dependencies to run the model
from transformers import AutoTokenizer, pipeline
from optimum.onnxruntime import ORTModelForTokenClassification
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

In [ ]:
# Create onnx model
tokenizerdis = AutoTokenizer.from_pretrained("elastic/distilbert-base-uncased-finetuned-conll03-english")
modeldis = ORTModelForTokenClassification.from_pretrained("elastic/distilbert-base-uncased-finetuned-conll03-english", export=True)

modeldis.save_pretrained('/save_directory')
tokenizerdis.save_pretrained('/save_directory')

In [ ]:
# create ORTQuantizer and define quantization configuration
quantizer = ORTQuantizer.from_pretrained(modeldis)
quantizer_config = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)

# apply the quantization configuration to the model and specify the directory to save the model
model_quantized_path = quantizer.quantize(
    save_dir="/model-quantized",
    quantization_config=quantizer_config,
)

In [13]:
def sanitise_data(text: str, entities: list) -> dict:
    return_entities=[]
    for entity in entities:
        santitise_entity={}
        santitise_entity['start'] = entity['start']
        santitise_entity['end'] = entity['end']
        santitise_entity['label'] = entity['entity_group']
        return_entities.append(santitise_entity)
    return {'text':text, 'ents':return_entities, 'title': None}

In [6]:
# Source: https://www.darcypattison.com/writing/revision/400-words-exactly/#:~:text=As%20a%20self%2Dtaught%20writer,LeGuin%2C%20the%20respected%20science%20fiction
text='As a self-taught writer who has taken the long, winding road towards writing and literary efforts, I was slow to learn about writing long sentences,\
 both how to do so and why one might want to do so, but finally was enlightened by three articulate authors and their books: Ursula LeGuin, the respected \
 science fiction and fantasy writer, author of The Left Hand of Darkness, and the popular children’s series, Wizard of the Earthsea, encourages long \
 sentences in her how-to book on writing, Steering the Craft, by quoting a 354-word sentence from Mark Twain’s Huckleberry Finn, which takes the long, \
 slow route – LeGuin calls it the “marvelously supple connections of complex syntax” – to describing the details of a sunrise over the Mississippi River\
 , including the sights, sounds, and smells of the unfolding morning; the second book which is less artistic, but perhaps more helpful to me personally \
 was Ann Longknife, Ph.D and K.D. Sullivan’s book, The Art of Styling Sentences: 20 Patterns for Success – look for the second edition published in 2002 \
 – which I studied with a writing friend, KN, and found to be extremely helpful in reviewing colons, semi-colons, appostives, etc, especially as KN and \
 I posted to mutual mailing lists and encourage each other to use the patterns correctly and creatively and learned that control of language was essential\
  to make the words mean what you want it to mean ( in fact, I found this book to be so useful, that I required it as a text when I taught Freshman \
  Composition); and third, was Dona Hickey’s wondeful book, Developing a Written Voice, a virtual gem of a book – it’s not for the faint-hearted, because\
   it reads like a college text book, but it’s a gem, nonetheless – which encourages the exploration of both long and short sentences, including sentence\
    fragments, while Hickey also gives the writer a range of options for creating coherence and cohesion among the various parts of the sentence, \
    including traditional rhetorical strategies such as schemes (unusual patterns of words) : schemes of balance, such as parallel structures, \
    antithesis and the isocolon; schemes of unusual or inverted word order, such as anastrophe and parenthesis; schemes of omission, such as ellipsis, \
    asyndeton or polysyndeton; schemes of repetition, such as alliteration, polyptoton, assonance, anaphora, epistrophe, epanalepsis, anadiplosis,\
     tricolon, chiasmus, and of course, long lists – all useful tools to create long sentences and keep them understandable. Writing long is fun. \
     Really. Try it.'

In [ ]:
tokenizerdis = AutoTokenizer.from_pretrained("/save_directory")
modeldis = ORTModelForTokenClassification.from_pretrained("/model-quantized")

#create transformers pipeline
onnx_ner = pipeline("token-classification", model=modeldis, tokenizer=tokenizerdis, aggregation_strategy='simple')

In [ ]:
# Function to test the time the model takes to run and return results
import time
for i in range(5):
    start_time=time.time()
    pred = onnx_ner(text)
    pred = sanitise_data(text, pred)
    end_time=time.time()
    print (f"The time to return results for 400 words of text is: {(end_time-start_time):.2f} seconds")